## DatetimeOutputParser
**날짜와 시간 정보를 파싱**하는 전용 파서입니다.

**주요 특징:**

- **다양한 날짜 형식 지원**: 여러 언어와 형식의 날짜 인식
- **자동 형식 감지**: 입력 형식을 자동으로 판단
- **datetime 객체 변환**: Python datetime 객체로 직접 변환
- **시간대 처리**: 타임존 정보 포함 처리 가능

**활용 사례:**

- 이벤트 일정 추출
- 문서에서 날짜 정보 파싱
- 로그 파일 시간 분석
- 일정 관리 시스템

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from datetime import datetime, timedelta

print(datetime.today())

2025-06-11 14:35:59.132287


In [4]:
# 출력 파서 초기화 (시간대 포함 가능)
datetime_parser = DatetimeOutputParser()
format_instructions = datetime_parser.get_format_instructions()

print("날짜 출력 형식 지시사항:")
print(format_instructions)

# 프롬프트 템플릿
# 현재 날짜를 명시적으로 프롬프트에 주입
current_date = datetime.now().strftime("%Y-%m-%d")
template = f"""
현재 날짜: {current_date}
다음 텍스트에서 날짜/시간 정보를 추출하세요. 상대적 표현(예: '다음 주 금요일')은 현재 날짜를 기준으로 계산합니다.
텍스트: {{text}}

{{format_instructions}}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)
print(prompt)

날짜 출력 형식 지시사항:
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1206-07-12T20:06:29.682137Z, 1412-08-11T00:26:06.176552Z, 1213-07-24T00:07:25.859283Z

Return ONLY this string, no other words!
input_variables=['text'] input_types={} partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1206-07-12T20:06:29.682137Z, 1412-08-11T00:26:06.176552Z, 1213-07-24T00:07:25.859283Z\n\nReturn ONLY this string, no other words!"} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template="\n현재 날짜: 2025-06-11\n다음 텍스트에서 날짜/시간 정보를 추출하세요. 상대적 표현(예: '다음 주 금요일')은 현재 날짜를 기준으로 계산합니다.\n텍스트: {text}\n\n{format_instructions}\n"), additional_kwargs={})]


In [5]:
# 모델 초기화 (temperature=0.1로 설정해 정확한 날짜 출력 강조)
#model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.1
)

# 테스트 데이터 (다양한 날짜 형식 포함)
texts = [
    "회의는 2025년 6월 15일 오후 2시에 예정되어 있습니다.",
    "프로젝트 마감일은 다음 주 금요일입니다.",
    "행사 시작: 7/25/2025 18:00 KST",
    "3일 후에 시스템 점검이 진행됩니다."
]

# 체인 실행 및 결과 출력
chain = prompt | model | datetime_parser

for text in texts:
    print(f"\n원본 텍스트: {text}")
    output = chain.invoke({"text": text})
    print(f"추출된 날짜: {output.strftime('%Y-%m-%d %H:%M:%S %Z')}")

    
# 이벤트 추출용 프롬프트
current_date = datetime.now().strftime("%Y-%m-%d")

event_template = """
현재 날짜: {current_date}
다음 텍스트에서 모든 이벤트의 날짜/시간을 추출하세요. 각 이벤트는 이름과 날짜를 포함해야 합니다.
텍스트: {text}

출력 형식:
- 이벤트명: [이름]
- 날짜: [YYYY-MM-DD HH:MM:SS]
"""

event_prompt = ChatPromptTemplate.from_template(event_template)
event_chain = event_prompt | model

# 예시 텍스트 (여러 이벤트 포함)
event_text = """
12월 10일에 크리스마스 마켓이 열리고, 12월 24일에는 크리스마스 이브 파티가 있습니다.
또한 1월 1일 00:00에 새해 카운트다운이 진행될 예정입니다.
"""

print(event_chain.invoke({"current_date":current_date, "text": event_text}).content)


원본 텍스트: 회의는 2025년 6월 15일 오후 2시에 예정되어 있습니다.
추출된 날짜: 2025-06-15 14:00:00 

원본 텍스트: 프로젝트 마감일은 다음 주 금요일입니다.
추출된 날짜: 2025-06-20 00:00:00 

원본 텍스트: 행사 시작: 7/25/2025 18:00 KST
추출된 날짜: 2025-07-25 18:00:00 

원본 텍스트: 3일 후에 시스템 점검이 진행됩니다.
추출된 날짜: 2025-06-14 00:00:00 
이 텍스트에서 이벤트 날짜/시간 추출:

- 이벤트명: 크리스마스 마켓
- 날짜: 2025-12-10 00:00:00

- 이벤트명: 크리스마스 이브 파티
- 날짜: 2025-12-24 00:00:00

- 이벤트명: 새해 카운트다운
- 날짜: 2025-01-01 00:00:00
